In [7]:
from bs4 import BeautifulSoup
import requests
import time
import datetime
import pandas as pd
import regex as re
from splinter import Browser

In [28]:
url = "https://www.wayfair.com/appliances/sb0/built-in-dishwashers-c1874332.html"

In [29]:
# Set up Splinter
browser = Browser('chrome')

In [30]:
# 
browser.visit(url)

In [32]:
# Parse the HTML
html = browser.html
html_soup = BeautifulSoup(html, 'html.parser')

# Start

In [50]:

results = html_soup.find_all(attrs={"data-hb-id": "Grid.Item", "class":"kzv0b81_6101"})

In [99]:
for result in results:
    title = result.find('h2', class_='kb51y90_6101 kb51y91_6101 kb51y93_6101')
    
    if title:
        
        # Grab the maker
        num_ratings_info = result.find(class_="_1xxktfua_6101 undefined")
        if num_ratings_info:
            num_ratings = num_ratings_info.text.replace("(", "").replace(")","")
        else:
            num_ratings=None
       
        print(num_ratings)

    print('---')  # Separation between each dishwasher

802
---
8155
---
3610
---
2213
---
161
---
955
---
210
---
331
---
199
---
796
---
1315
---
2200
---
523
---
1401
---
115
---
39
---
108
---
1445
---
5192
---
184
---
2868
---
6
---
3
---
1403
---
3
---
1
---
3
---
705
---
703
---
2
---
10
---
None
---
1
---
101
---
1
---
14
---
48
---
8
---
9
---
7
---
9
---
1
---
16
---
8
---
67
---
None
---
5
---
None
---
---
---
---
---
---
---
---
---


In [100]:
dishwashers = []
for result in results:
    dw_info = {}
    # Extract the product rating
    title = result.find('h2', class_='kb51y90_6101 kb51y91_6101 kb51y93_6101')
    
    if title:
        
        # Grab the maker
        dw_info["maker"] = result.find(class_="oakhm610g_6101 oakhm614l_6101 nhya890_6101").text

        #Grab the size
        size = re.findall(r'[0-9]+"|[0-9]+ in', title.text)
        if size:
            dw_info["size"]=size[0][:2]
        else:
            dw_info["size"]=None

        # Grab noise level; looked it up and all dishwashers have a noiselevel between 30-100 dBA
        noise_level = re.findall(r"[0-9]+ dBA|[0-9]+ Decibel dBA", title.text)
        if noise_level:
            dw_info["noise_level"]=noise_level[0][:2]
            
        else:
            dw_info["noise_level"] = None

        # Grab the rating
        rating_info = result.find(class_="_1xxktfu7_6101 _1xxktfu3_6101 _1xxktfu8_6101")
        if rating_info:
            rating_style = re.findall(r"width:[0-9]+|width: [0-9]+", rating_info["style"]) 
            dw_info["rating"]=int(rating_style[0][-2:])/20
        else: 
            dw_info["rating"] =None

        # Grab the number of ratings:
        num_ratings_info = result.find(class_="_1xxktfua_6101 undefined")
        if num_ratings_info:
            dw_info["num_ratings"] = num_ratings_info.text.replace("(", "").replace(")","")
        else:
            dw_info["num_ratings"]=None

        # Add the dishwasher to the list
        dishwashers.append(dw_info)

  

In [101]:
pd.DataFrame(dishwashers)

,maker,size,noise_level,rating,num_ratings
0,GE Appliances,24,55,4.0,802
1,GE Appliances,24,48,4.5,8155
2,GE Appliances,24,46,4.5,3610
3,GE Appliances,24,59,3.5,2213
4,EdgeStar,18,52,4.5,161
5,GE Profile™,24,39,4.5,955
6,ZLINE,18,None,4.5,210
7,GE Profile™,None,47,4.0,331
8,GE Appliances,24,51,3.5,199
9,GE Profile™,24,45,4.5,796


# Start parsing

In [27]:
browser.quit()